# Data Inspection — Primeiro Contato com o Dataset

Este notebook tem como objetivo realizar o **primeiro contato estruturado** com o dataset
de atrasos de voos, validando sua integridade, coerência operacional e principais
características estatísticas.

Neste estágio, **nenhuma transformação, limpeza ou feature engineering é realizada**.
O foco é exclusivamente **observação e entendimento**, servindo de base para decisões
futuras de modelagem.


In [17]:
from src.data.load import load_raw_data
import sys
from pathlib import Path
import pandas as pd

# adiciona a raiz do projeto ao path
sys.path.append(str(Path("..").resolve()))


# carrega o dataframe bruto
df = load_raw_data()

In [18]:
# Informações básicas sobre o dataframe
print("Informações do DataFrame:")
print(df.info())
print("\nPrimeiras linhas do DataFrame:")
print(df.head(2))

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318017 entries, 0 to 318016
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   year                 318017 non-null  int64  
 1   month                318017 non-null  int64  
 2   carrier              318013 non-null  object 
 3   carrier_name         318013 non-null  object 
 4   airport              318014 non-null  object 
 5   airport_name         318017 non-null  object 
 6   arr_flights          317524 non-null  float64
 7   arr_del15            317285 non-null  float64
 8   carrier_ct           317525 non-null  float64
 9   weather_ct           317523 non-null  float64
 10  nas_ct               317529 non-null  float64
 11  security_ct          317529 non-null  float64
 12  late_aircraft_ct     317529 non-null  float64
 13  arr_cancelled        317529 non-null  float64
 14  arr_diverted         317527 non-null  floa

In [19]:
# Estatísticas descritivas
print(df.describe())

                year          month    arr_flights      arr_del15  \
count  318017.000000  318017.000000  317524.000000  317285.000000   
mean     2012.450957       6.497844     381.766670      72.905076   
std         5.678296       3.459423    1027.156722     198.936754   
min      2003.000000       1.000000       1.000000       0.000000   
25%      2007.000000       3.000000      59.000000       9.000000   
50%      2012.000000       6.000000     120.000000      23.000000   
75%      2018.000000      10.000000     273.000000      56.000000   
max      2022.000000      12.000000   21977.000000    6377.000000   

          carrier_ct     weather_ct         nas_ct    security_ct  \
count  317525.000000  317523.000000  317529.000000  317529.000000   
mean       21.073149       2.616407      24.005228       0.179037   
std        47.671580       9.968640      85.113757       0.844834   
min         0.000000       0.000000      -0.010000       0.000000   
25%         3.000000       0.0000

In [20]:
# Estatisticas para colunas categóricas
print(df.describe(include=['object', 'category']))

       carrier           carrier_name airport  \
count   318013                 318013  318014   
unique      29                     33     420   
top         OO  SkyWest Airlines Inc.     DTW   
freq     38145                  38145    3022   

                                   airport_name  
count                                    318017  
unique                                      420  
top     Detroit, MI: Detroit Metro Wayne County  
freq                                       3022  


In [21]:
# Contagem de valores nulos por coluna e percentual
null_counts = df.isnull().sum()
null_percent = (null_counts / len(df)) * 100
null_summary = pd.DataFrame({'null_count': null_counts, 'null_percent': null_percent})
print("\nContagem e percentual de valores nulos por coluna:")
print(null_summary)


Contagem e percentual de valores nulos por coluna:
                     null_count  null_percent
year                          0      0.000000
month                         0      0.000000
carrier                       4      0.001258
carrier_name                  4      0.001258
airport                       3      0.000943
airport_name                  0      0.000000
arr_flights                 493      0.155023
arr_del15                   732      0.230176
carrier_ct                  492      0.154709
weather_ct                  494      0.155338
nas_ct                      488      0.153451
security_ct                 488      0.153451
late_aircraft_ct            488      0.153451
arr_cancelled               488      0.153451
arr_diverted                490      0.154080
arr_delay                   494      0.155338
carrier_delay               492      0.154709
weather_delay               488      0.153451
nas_delay                   488      0.153451
security_delay              

## Variáveis temporais

O dataset contém variáveis temporais explícitas (`year`, `month`), representando
agregações mensais por aeroporto e companhia aérea.

A granularidade temporal é **mensal**, o que sugere que qualquer modelagem futura
deverá considerar:
- dependência temporal
- risco de data leakage em splits aleatórios
- necessidade de validação temporal adequada


## Target — entendimento inicial

O target do projeto é definido conceitualmente (em `config.py`) e não apenas herdado
do dataset.

Nesta etapa, busca-se apenas verificar:
- se o target está presente no dataset
- sua escala e variabilidade
- se ele pode ser consequência direta de outras colunas, levantando suspeitas
de possível data leakage


## Observações e riscos identificados

- As variáveis de atraso apresentam forte assimetria à direita, com outliers severos,
  comportamento esperado em sistemas logísticos.
- Algumas variáveis (`nas_delay`, `nas_ct`) apresentam valores negativos de pequena
  magnitude, cuja interpretação depende da definição semântica oficial.
- A presença de colunas que representam decomposição do atraso total levanta forte
  suspeita de **data leakage**, exigindo análise cuidadosa antes da modelagem.
- O percentual de valores ausentes é baixo, mas seu padrão precisa ser melhor
  compreendido antes de qualquer decisão de limpeza.


## Escopo deste notebook

Este notebook se limita à **inspeção inicial dos dados**.
Não são realizadas transformações, limpezas ou análises exploratórias visuais.

Próximos passos incluem:
- análise conceitual mais profunda das variáveis
- EDA visual direcionada
- definição clara de features válidas para modelagem
